In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

import sys

Using TensorFlow backend.


In [2]:
sys.path.append('/home/bence/Documents/vitmav45-2016-train-validate-test-repeat/data_preparation')
import preprocess as ppx

In [7]:
# a képek betöltése és előfeldolgozása
image_file_names, data = ppx.csv_load()
image_name_dict, images = ppx.load_images(image_file_names)
train_images, train_data = ppx.data_preprocess(images, data, image_name_dict)

[[[161 189 193]
  [161 189 193]
  [162 190 194]
  ..., 
  [147 192 215]
  [147 192 215]
  [150 195 218]]

 [[167 194 201]
  [164 193 199]
  [162 189 196]
  ..., 
  [150 195 218]
  [142 187 210]
  [145 190 213]]

 [[166 195 203]
  [163 193 201]
  [159 188 196]
  ..., 
  [137 182 205]
  [138 183 206]
  [146 191 214]]

 ..., 
 [[150 116  79]
  [159 127  89]
  [184 150 115]
  ..., 
  [144 152  92]
  [152 163 105]
  [142 155 102]]

 [[185 152 117]
  [175 145 109]
  [185 152 117]
  ..., 
  [141 144  89]
  [154 158 107]
  [132 140  93]]

 [[177 147 111]
  [167 137 101]
  [178 148 112]
  ..., 
  [153 151 100]
  [170 172 123]
  [137 142 101]]]
[[[53 60 68]
  [51 58 66]
  [49 56 64]
  ..., 
  [71 81 83]
  [75 85 87]
  [79 89 91]]

 [[48 55 63]
  [50 57 65]
  [50 57 65]
  ..., 
  [73 83 85]
  [75 85 87]
  [77 87 89]]

 [[45 52 60]
  [48 55 63]
  [51 58 66]
  ..., 
  [77 87 88]
  [75 85 86]
  [75 85 86]]

 ..., 
 [[80 69 75]
  [63 52 58]
  [45 34 40]
  ..., 
  [69 58 64]
  [66 55 59]
  [64 53 57]]

In [10]:
imgs = np.array(train_images)

In [8]:
arr = np.array([np.asarray(ls[1]) for ls in train_images])
labels = np.array([ls[1] for ls in train_data])

encoder = preprocessing.LabelEncoder()
encoder.fit(labels)
encoded_l = encoder.transform(labels)

labels_onehot = to_categorical(encoded_l)

In [3]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [4]:
# epoch szám
nb_epoch=100

In [19]:
# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót
x = base_model.output
x = GlobalAveragePooling2D()(x)
# ezután hozzáadunk egy előrecsatolt réteget ReLU aktivációs függvénnyel
x = Dense(1024, activation='relu')(x)
# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='sigmoid')(x)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

tf


In [20]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='binary_crossentropy')

In [21]:
labels_onehot.shape[1]

459

In [23]:
model.fit(imgs, labels_onehot, batch_size=6, nb_epoch=10, callbacks=[history])

Epoch 1/10
770/770 [==============================] - 19s - loss: 0.0065    
Epoch 2/10
770/770 [==============================] - 19s - loss: 0.0040    
Epoch 3/10
770/770 [==============================] - 19s - loss: 0.0025    
Epoch 4/10
770/770 [==============================] - 19s - loss: 0.0019    
Epoch 5/10
770/770 [==============================] - 19s - loss: 0.0013    
Epoch 6/10
770/770 [==============================] - 19s - loss: 0.0011    
Epoch 7/10
770/770 [==============================] - 20s - loss: 9.9492e-04    
Epoch 8/10
770/770 [==============================] - 19s - loss: 9.3049e-04    
Epoch 9/10
770/770 [==============================] - 19s - loss: 8.7230e-04    
Epoch 10/10
770/770 [==============================] - 19s - loss: 8.0583e-04    


In [24]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

Az Inception V3 konvolúciós rétegei:
0 input_2
1 convolution2d_95
2 batchnormalization_95
3 convolution2d_96
4 batchnormalization_96
5 convolution2d_97
6 batchnormalization_97
7 maxpooling2d_4
8 convolution2d_98
9 batchnormalization_98
10 convolution2d_99
11 batchnormalization_99
12 maxpooling2d_5
13 convolution2d_103
14 batchnormalization_103
15 convolution2d_101
16 convolution2d_104
17 batchnormalization_101
18 batchnormalization_104
19 averagepooling2d_11
20 convolution2d_100
21 convolution2d_102
22 convolution2d_105
23 convolution2d_106
24 batchnormalization_100
25 batchnormalization_102
26 batchnormalization_105
27 batchnormalization_106
28 mixed0
29 convolution2d_110
30 batchnormalization_110
31 convolution2d_108
32 convolution2d_111
33 batchnormalization_108
34 batchnormalization_111
35 averagepooling2d_12
36 convolution2d_107
37 convolution2d_109
38 convolution2d_112
39 convolution2d_113
40 batchnormalization_107
41 batchnormalization_109
42 batchnormalization_112
43 batchnorma

In [25]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(np.asarray(train_images), labels_onehot,  batch_size=6, nb_epoch=10, callbacks=[history])
print("Tanítás vége.")

Epoch 1/10
770/770 [==============================] - 28s - loss: 8.1555e-04    
Epoch 2/10
770/770 [==============================] - 24s - loss: 8.1458e-04    
Epoch 3/10
770/770 [==============================] - 24s - loss: 7.0595e-04    
Epoch 4/10
770/770 [==============================] - 24s - loss: 7.4208e-04    
Epoch 5/10
770/770 [==============================] - 24s - loss: 5.9879e-04    
Epoch 6/10
770/770 [==============================] - 24s - loss: 8.5563e-04    
Epoch 7/10
770/770 [==============================] - 24s - loss: 7.7508e-04    
Epoch 8/10
770/770 [==============================] - 25s - loss: 7.0024e-04    
Epoch 9/10
770/770 [==============================] - 24s - loss: 7.3867e-04    
Epoch 10/10
770/770 [==============================] - 24s - loss: 6.8023e-04    
Tanítás vége.


In [36]:
predicted_values = model.predict(np.asarray(train_images))

In [57]:
encoder.inverse_transform(max_pred).shape

(770, 459)

In [51]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
print()

In [ ]:
ecovered_X = np.array([ohc.active_features_[col] for col in out.sorted_indices().indices])
                .reshape(n_samples, n_features) - ohc.feature_indices_[:-1]

In [50]:
np.zeros((3,4), dtype=int)

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [39]:
xx = np.array([4,56,67.3,7])

In [59]:
predicted_values.shape

(770, 459)